In [1]:
import sys
import os
import cobra
import libsbml
from cobra import Model, Reaction, Metabolite, manipulation, test
import pandas as pd
import csv
import copy
from pprint import pprint
from tabulate import tabulate



#Change working dir first, ty ChatGPT, much loves
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src")

import model_manipulation  as mm



In [14]:
ios2164_orig = cobra.io.read_sbml_model('./model/ios2164_orig.xml')

ios2164_orig
inf = 1e6

In [15]:
#Check mass balance of original ios2164 model

imbalanced_rxns = list()

non_exchange_rxns = set(ios2164_orig.reactions) - set(ios2164_orig.boundary) - set(ios2164_orig.demands)

for rxns in non_exchange_rxns:
    #Check if rxns are not in boundary
        #Retrieve mass balance dictionary
        mb_dict = rxns.check_mass_balance()
        #Check if imbalanced + if it has H+
        if len(mb_dict) != 0:
            imbalanced_rxns.append(rxns)
        

#149 imbalanced rxns that are in boundary
#636 reactions that are within the cell. Let's check which of these are imbaalanced.
print('initial imbalanced reactions length for ios2164_orig:', len(imbalanced_rxns))
print('percent imbalanced reactions', (len(imbalanced_rxns)/len(non_exchange_rxns)) * 100)

initial imbalanced reactions length for ios2164_orig: 647
percent imbalanced reactions 28.09379070777247


In [17]:
def rebalance_model(model):
    '''
    This function is used to rebalance any reactions
    found in the old model to reduce any possible 
    futile cycling. This curation step is done with the aid of Metacyc and BRENDA
    as well as the check_mass_balance() function from CobraPy
    to validate any inconsistent chemical equations as well as
    compound formulas that weren't encoded properly. The only remaining 
    reactions that weren't curated were as  follows:
    - reactions with undefined side groups encoded as "R" and were therefore expectedly
    unbalanced since CobraPy does not detect them
    - Charge imbalances that were validated and checked for any inconsistencies
    '''


#     ios2164_orig.metabolites.cdpdag_os_r.formula = 'C14H18N3O15P2R2'
#     ios2164_orig.metabolites.dhqrctn_c.formula = 'C15H11O7'
    ios2164_orig.metabolites.oopc4coa_x.formula = 'C35H52N7O19P3S'
    ios2164_orig.metabolites.oopc4coa_x.formula = 'C35H52N7O19P3S'
#     ios2164_orig.metabolites.udpkrmn_c.formula = 'C15H22N2O16P2'
  
    
    ios2164_orig.metabolites.udprmn_c.formula = 'C15H22N2O16P2' #Works
    ios2164_orig.metabolites.cgly_c.formula = 'C5H10N2O3S' #Not encoded properly
    ios2164_orig.metabolites.cholp_r.formula = 'C5H13NO4P'
    ios2164_orig.metabolites.cholp_s.formula = 'C5H13NO4P'
    ios2164_orig.metabolites.dtmp_s.formula = 'C10H13N2O8P' #Was not encoded properly
    ios2164_orig.metabolites.get_by_id('5dpmev_c').formula = 'C6H10O10P2'
    ios2164_orig.metabolites.lnlncoa_r.formula = 'C39H62N7O17P3S'
    ios2164_orig.metabolites.lnlnlACP_s.formula = 'C29H52N2O8PRS'
    ios2164_orig.metabolites.lnlnl_c.formula = 'C18H29O2'
    ios2164_orig.metabolites.lnlnl_s.formula = 'C18H29O2'
    ios2164_orig.metabolites.lnlnl_x.formula = 'C18H29O2'
    ios2164_orig.metabolites.o2_u.formula = 'O2' #Not encoded properly
    ios2164_orig.metabolites.plast_s.formula = 'C53H82O2'
    
    
    ios2164_orig.reactions.AASADc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.AATCm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.AATCs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ACDOc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACGAMKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACHBSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ACHYSLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACRSPc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ACSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ADEDAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ADMATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ADPARTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ALATm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ALATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ALTNDAc.add_metabolites({'h_c':-2.0})
    ios2164_orig.reactions.AMETAHYCSTc.add_metabolites({'h_c':1,  'h2o_c':1})
    ios2164_orig.reactions.ANTPPTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ANXANDEOs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.AOBUTDx.add_metabolites({'h_x':1})
    ios2164_orig.reactions.APPRTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ARGSSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ARGTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ARGTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASCOXc.add_metabolites({'h_c':8})
    ios2164_orig.reactions.ASCPO1c.add_metabolites({'h_c':4})
    ios2164_orig.reactions.ASCPO1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.GTHDHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ASCPO2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ASCPO2s.add_metabolites({'h_s':3})
    ios2164_orig.reactions.ASNS1s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASNTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASNtm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ASPALs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASPCLc.add_metabolites({'h_c': -1})
    ios2164_orig.reactions.ASPCT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ASPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASPHGNAHr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.ASPHGNAr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.ASPTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ASPTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ATAHc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ATPAPc.reaction = 'atp_c + 2.0 h2o_c --> amp_c + 2.0 pi_c + 2.0 h_c'
    ios2164_orig.reactions.ATPSs.add_metabolites({'h_u':-1})
    ios2164_orig.reactions.BRS2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.C3STDH2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.C4STMO1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CAFFMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CAVNS2s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.CBMKs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.CBPSs.add_metabolites({'h_s':3})
    ios2164_orig.reactions.CC3H1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CDPDAGSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.CELLSc.add_metabolites({'h_c': 2})
    ios2164_orig.reactions.CHALIc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CHOLPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.CKDH1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKDH2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKDH3c.add_metabolites({'h_c':1})
    
    ios2164_orig.reactions.CKGT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT4c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT5c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT6c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT7c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT8c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKPc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.CMPAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CNDNG3Gc.add_metabolites({'h_c': 2})
    ios2164_orig.reactions.CNDNG5Gc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.CNDNGLT5c.add_metabolites({'h_c':2})
    ios2164_orig.reactions.COUMGUTFc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.COUMNSc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CTPCHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CYSSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.CYSTLm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.CYTTHs.bounds = (-inf, 0)
    ios2164_orig.reactions.DAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.DCTPDAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.DGSNTTPc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.DHDHc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.DHDPSs.add_metabolites({'h_s': -2})
    ios2164_orig.reactions.DHKFORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.DHKRc.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.DHNCOAHs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DHNCOASs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.DHNOPTs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHNPPTs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.DHORSts.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.DHPDc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.DHPRs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHPRys.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHPSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.DHQORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.DHQRc.add_metabolites({'h2o_c':-1})
    ios2164_orig.reactions.DMATTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.DMPPTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DMPQMTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DNTPPAc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.EAR160ym.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.EAR160ys.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ECASDATEO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDATEO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.EIKAUROc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.EIKAURSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.EKAURSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ENTBSs.delete #Remove bacterial reaction w/ no GPR
    ios2164_orig.reactions.ESANDO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ESANDSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ESANDSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ETHAPTr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA140COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA160ACOASc.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.FA160ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA160COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA161COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA180COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA181COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA182ACOASc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FA182ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA182COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA183ACOASc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FA183ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA183COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA200COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA220COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA240COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA260COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA280COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA300COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FAS161COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS181ACP_Lm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.FAS181ACP_Ls.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.FAS181COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS182ACP_Ls.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.FAS182COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS183ACP_Ls.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FAS183COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FERHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FGFTm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.FGFTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FMNATc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FMNATm.add_metabolites({'h_m':2})
    ios2164_orig.reactions.FMNATs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.FNORs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.FORMHLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FRDPFTs.add_metabolites({'h_s': 5})
    ios2164_orig.reactions.FRPTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FRTTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FTHFCLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FTHFCLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.FTHFCLs.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GA4DOc.add_metabolites({'h2o_c':1, 'h_c':1})
    ios2164_orig.reactions.GAAKORG10c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG11c.add_metabolites({'h2o_c':1})
    ios2164_orig.reactions.GAAKORG12c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG8c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG9c.add_metabolites({'h2o_c':1})
    ios2164_orig.reactions.GAMPTc.add_metabolites({'h_c':1.0})
    ios2164_orig.reactions.GBDPs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GCOADCx.add_metabolites({'h_x':-1})
    ios2164_orig.reactions.GDPMDHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GENT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GENT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GLCNSc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.GLUTRSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GLYATx.add_metabolites({'h_x':-1})
    ios2164_orig.reactions.GLYB1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.GLYDHDm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.GLYTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GLYTLs.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GMPc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GNSKc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GRTTm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GRTTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GSNPc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GSNRHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GTHDHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GTHDHs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.GTPCYTDPTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GTPCYTDPTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GTPDHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GTPDPKs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.H2Sts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.HCYSMT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.HDHs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.HEMEOSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HFERMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.HISTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HISTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.HMGCOARc.add_metabolites({'h_c': -1.0})
    ios2164_orig.reactions.HMGCOARyc.add_metabolites({'h_c': -1.0})
    ios2164_orig.reactions.HMGLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HOXGs.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.HPPKm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.IAAAMYHc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.IAAIASc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.IAAIGSc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ILETLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ILETLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.IPDPT1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.IPDPT2s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.IPKK2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK4c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK5c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK6c.add_metabolites({'h_c': -2.0})
    ios2164_orig.reactions.ITK1Kc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.LCYCORc.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.LCYSTSm.add_metabolites({'h_m':2})
    ios2164_orig.reactions.LCYSTSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.LIMSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.LNL9HPLs.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.LPLGDOc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.LYSTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.LYSTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MACPCLm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MACPCLs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MACPMMEc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MANKINc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MARMSc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MCOATAc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MCOATAm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.MCOATAs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MDHARm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MDHARyc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MDHARym.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MECDPDH5s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MEPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.METSc.add_metabolites({'h_c':1.0})
    ios2164_orig.reactions.METSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.METTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.METTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MGCHLs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.MGCm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.MI1345PP2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1345PPc.add_metabolites({'h_c': -2.0})
    ios2164_orig.reactions.MI1346PK1c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1346PK2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1456PKc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI34Pc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MLTHFR_Fc.add_metabolites({'h_c':3})
    ios2164_orig.reactions.MLTHFRc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MLTHFRs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MOMASs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MOMASys.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MPMECY1s.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MPMECY2s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MPMTs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.MRs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MSBQMTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MTAMnh4m.add_metabolites({'h_m':1})
    ios2164_orig.reactions.NADOR2u.add_metabolites({'h_u':-1.0})
    ios2164_orig.reactions.NNAMc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.NNDPRs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.NO3Rm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.NOSc.reaction = '2 arg-L_c + 3 nadph_c + 4 o2_c --> 2 citr-L_c + 3 nadp_c + 2 no_c + 4 h2o_c'
    ios2164_orig.reactions.NRGNAORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.NRGNMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.NRGNNORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.OCTDPOTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.ORPRTs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.OXAACc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO1c.add_metabolites({'h_c':- 1})
    ios2164_orig.reactions.P450MO2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO4c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO5c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO6c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PCAHr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.PEDCm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PEDCr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.PEPSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PHENH4Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PHETLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PHETLs.add_metabolites({'h_s':1}) 
    ios2164_orig.reactions.PHYTS1s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PHYTS2s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PINCLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PLIBTNSm.add_metabolites({'h_m':-3})
    ios2164_orig.reactions.PLIVLPNSm.add_metabolites({'h_m':-3})
    ios2164_orig.reactions.PPASm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.PPASs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.PPDKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.PPDKs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PPPGPPPc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.PRADPs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.PRAICs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.PRDPARs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRFGSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRIMADNSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRIMARTSc.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PROTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PROTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.QRCTNGLT3c.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.RBPCs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.RIBFKc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.RIBFSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.SELNPS.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.SERTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.SERTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SHCHFs.add_metabolites({'h_s':3})
    ios2164_orig.reactions.SOXs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SQTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.STEMOc.add_metabolites({'h_c': -1})
    ios2164_orig.reactions.STEMSs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.SUATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SULRs.add_metabolites({'h_s':6})
    ios2164_orig.reactions.TAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.THR3DHx.add_metabolites({'h_x':-1.0})
    ios2164_orig.reactions.THRTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TMDPKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TMPPPs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TRICINS1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TRICINS2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TRPTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TRPTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TYRLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TYRNH4Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.TYRTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TYRTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UAMASc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.UDPGALMc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.UDPGALts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UDPGATc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.UDPGts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UDPSQSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.UPRTs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.URDGLAHc.add_metabolites({'h_c':-1})     
    ios2164_orig.reactions.UREASEc.add_metabolites({'h_c':-2.0})
    ios2164_orig.reactions.UROc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.VALTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.VALTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.VLXANDEOs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.XSc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.get_by_id('2HEDBT8GTc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2HNDBT8GTc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2HNDGDHc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2SBZCOALs').add_metabolites({'h_s': 1})
    ios2164_orig.reactions.get_by_id('3OAS180m').add_metabolites({'h_m':1})
    ios2164_orig.reactions.get_by_id('3OAS180s').add_metabolites({'h_s':1})
    ios2164_orig.reactions.get_by_id('SUCL(ADP)m').add_metabolites({'h_m':-1})
    ios2164_orig.reactions.get_by_id('SUCL(GDP)m').add_metabolites({'h_m':-1})

    
    #Last iteration
    ios2164_orig.reactions.ACSERLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PRIMARDSs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DLDLBTs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.PC2Ds.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.GSNDc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.CYSTLs.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.FRDPFTm.add_metabolites({'h_m':5})
    
    
#     ios2164_orig.metabolites.cdpdag_os_r.formula = 'C14H18N3O15P2R2'
#     ios2164_orig.metabolites.ocdcya_s.formula = 'C18H29O2' #Doesn't work

In [34]:
ios2164_orig.reactions.CAVNS2s.check_mass_balance()

{'charge': -2.0, 'H': -1.0}

In [35]:
ios2164_orig.reactions.CAVNS2s

Reaction identifier,CAVNS2s
Name,argininosuccinate synthase
Memory address,0x07fb85be81760
Stoichiometry,asp-L_s + atp_s + uhom-L_s --> amp_s + cavsucc_s + ppi_s L-Aspartate + ATP + O-ureidohomoserine --> AMP + Canavaninosuccinate + Diphosphate
GPR,LOC_Os12g13320 or LOC_Os11g19770
Lower bound,0.0
Upper bound,1000000000.0


In [20]:
ios2164_orig.reactions.ngam_nadphox_m.reaction = '2 h_m + 2 nadph_m + o2_m --> 2 h2o_m + 2 nadp_m'
ios2164_orig.reactions.ngam_nadphox_s.reaction = '2 h_s + 2 nadph_s + o2_s --> 2 h2o_s + 2 nadp_s'
ios2164_orig.reactions.ngam_nadphox_c.reaction = '2 h_c + 2 nadph_c + o2_c --> 2 h2o_c + 2 nadp_c'


#This should be the formula for NGAM

In [36]:
imbalanced_rxns = list()


non_exchange_rxns = set(ios2164_orig.reactions) - set(ios2164_orig.boundary) - set(ios2164_orig.demands)


#Check mass balance of original ios2164 model
with ios2164_orig as ios2164_orig_copy:
    rebalance_model(ios2164_orig_copy)
#     ios2164_orig_copy.reactions.DHDHc.add_metabolites({'h_c':2})
    for rxns in non_exchange_rxns:
            #Retrieve mass balance dictionary
            mb_dict = rxns.check_mass_balance()
            #Check if imbalanced 
            if len(mb_dict) != 0:
                imbalanced_rxns.append(rxns)
                

#149 imbalanced rxns that are in boundary
#636 reactions that are within the cell. Let's check which of these are imbaalanced.
print('imbalanced reactions length for ios2164_orig after rebalancing:', len(imbalanced_rxns))
print('percent imbalanced reactions', (len(imbalanced_rxns)/len(non_exchange_rxns)) * 100)

imbalanced reactions length for ios2164_orig after rebalancing: 308
percent imbalanced reactions 13.37386018237082


In [38]:
ios2164_orig.reactions.MSx

Reaction identifier,MSx
Name,"5-methyltetrahydropteroyltriglutamate-homocysteine S-methyltransferase, peroxisomal"
Memory address,0x07fb85c8e0ac0
Stoichiometry,5mthglu_x + hcys-L_x --> met-L_x + thglu_x 5-Methyltetrahydropteroyltri-L-glutamate + L-Homocysteine --> L-Methionine + Tetrahydropteroyl-tri-L-glutamate
GPR,LOC_Os12g42876 or LOC_Os12g42884
Lower bound,0.0
Upper bound,1000000000.0


In [33]:
ios2164_orig.metabolites.get_by_id('hcys-L_x')

Metabolite identifier,hcys-L_x
Name,L-Homocysteine
Memory address,0x07fb85cc92340
Formula,C4H9NO2S
Compartment,x
In 1 reaction(s),MSx


In [22]:
for items in imbalanced_rxns:
    print(items.id,items.check_mass_balance())


ngam_atp_s {'charge': -1.0, 'H': 1.0, 'O': 1.0}
FA182ACPHs {'charge': -1.0, 'H': -3.0}
SKMHCTc {'H': 2.0}
RBPOs {'charge': -1.0, 'O': -1.0, 'H': -3.0}
EX_phloem {'charge': 0.1296, 'C': -62.107749999999996, 'H': -110.59700000000001, 'N': -1.7961599999999998, 'O': -53.50195}
MSx {'H': 4.0}
FA183ACPHs {'charge': -1.0, 'H': -3.0}
AMETAHYCSTc {'charge': -1.0, 'C': -1.0, 'H': -3.0}
PLAr {'charge': 1.0, 'H': 2.0}
DM_Phloem {'charge': 0.1296, 'C': -62.107749999999996, 'H': -110.59700000000001, 'N': -1.7961599999999998, 'O': -53.50195}
PLCr {'charge': 2.0, 'H': 1.0, 'O': -3.0, 'P': -1.0}
ngam_atp_m {'charge': -1.0, 'H': 1.0, 'O': 1.0}
ngam_atp_x {'charge': -1.0, 'H': 1.0, 'O': 1.0}
SIAT9c {'R': -2.0}
CAVNS2s {'charge': -2.0, 'H': -1.0}
G16MT2c {'C': -6.0, 'H': -10.0, 'O': -5.0}
DVCHLOR550s {'charge': 0.2481099999999999, 'X': -0.24811, 'H': 0.7443300000000015, 'N': -2.220446049250313e-16, 'O': 2.220446049250313e-16}
DVCHLOR530s {'charge': 0.20800000000000005, 'X': -0.208, 'H': 0.6240000000000006

In [39]:
rev_rxn =  list()

for rxns in non_exchange_rxns:
    if rxns.lower_bound != 0 and rxns.upper_bound != 0:
        rev_rxn.append(rxns)

In [43]:
for i in rev_rxn:
    if len(i.check_mass_balance()) != 0:
        print(i.name)

ATP NGAM for Chloroplast (Plastid)
S-adenosyl-L-methionine:L-histidine N-methyltransferase
ATP NGAM for Mitochondria
ATP NGAM for Peroxisome
aspartate aminotransferase, cysteate forming, plastidic
5-formyltetrahydrofolate cyclo-ligase, mitochondrial
Heme oxygenase (decyclizing)
coproporphyrinogen I oxidase
aspartate aminotransferase, cysteate forming, mitochondrial
2-oxoadipate dehydrogenase E2 component
ACP(Aerobic composition)
Fatty acid (Aerobic composition)
Acyl-CoA (Aerobic composition)
Acyl-CoA (Aerobic composition)
spectral decomposition of red LED (Mix of R, G & B)
uroporphyrinogen I reductase
Aminomethyltransferase
Phosphoribosylaminoimidazole carboxylase
cis-zeatin synthesis, 1
malonyl-CoA:[acyl-carrier-protein] S-malonyltransferase
Allantoicase
precorrin 2 dehydrogenase
Phenylalanine ammonia-lyase
selenophosphate synthase
Dihydropyrimidinase
Dihydropyrimidine dehydrogenase (NADP+)
Ethanolaminephosphotransferase, endoplasmic reticular
phosphatidate cytidylyltransferase, mitoc

In [11]:
ios2164_orig.metabolites.dhn_s

Metabolite identifier,dhn_s
Name,"1,4-Dihydroxy-2-naphthoate"
Memory address,0x07fb8a4844310
Formula,C11H7O4
Compartment,s
In 3 reaction(s),"DHNOPTs, DHNCOAHs, DHNPPTs"


In [10]:
ios2164_orig.reactions.dhn_s

AttributeError: DictList has no attribute or entry dhn_s

In [ ]:
ios2164_2cell = cobra.io.read_sbml_model('./model/ios2164_2cell.xml')
ios2164_2cell.reactions.DM_Phloem_BS

#I need to modify the objective function since Starch is being exported from M cell to BS cell. 

In [ ]:
at_core_model = cobra.io.read_sbml_model('./Supplementary_PrevLit/Blaetke & Brautigam/data/2018-23-05-mb-genC3.sbml')
internal_rxns = set(at_core_model.reactions) - set(at_core_model.boundary)

unbalanced_rxns = list()
for rxns in internal_rxns:
    if len(rxns.check_mass_balance()) != 0:
#         print(rxns.id, '\t', rxns.check_mass_balance())
        unbalanced_rxns.append(rxns)
    
    
at_core_model
    

In [ ]:
print('internal reactions in B&B core AT model: ', len(internal_rxns))
print('unbalanced rxns in B&B: ', len(unbalanced_rxns))
print('percent unbalanced: ', len(unbalanced_rxns)/len(internal_rxns) * 100)

#IN the case of B&B's model puro charge imbalance yung nagaganap. Yung sa case ko kasi karamihan is mass imbalance talaga.

In [23]:
#How about yung core Arabidopsis model nina Shameer et al (2019) -- yung C3 mitochondria
shameer_model = cobra.io.read_sbml_model('./Supplementary_PrevLit/Shameer (2019)/core_model.xml')
internal_rxns = set(shameer_model.reactions) - set(shameer_model.boundary)


unbalanced_rxns = list()
for rxns in internal_rxns:
    if len(rxns.check_mass_balance()) != 0:
#         print(rxns.id, '\t', rxns.check_mass_balance())
        unbalanced_rxns.append(rxns)
    
print('internal reactions in Shameer core AT model: ', len(internal_rxns))
print('unbalanced rxns in Shameer AT model: ', len(unbalanced_rxns))
print('percent unbalanced: ', len(unbalanced_rxns)/len(internal_rxns) * 100)

#IN the case of B&B's model puro charge imbalance yung nagaganap. Yung sa case ko kasi karamihan is mass imbalance talaga.

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_FUM_p "FUM">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_FUM_p "FUM">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_FUM_m "FUM">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_FUM_m "FUM">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_VAL_c "VAL">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_VAL_c "VAL">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_2_KETOGLUTARATE_c "2-KETOGLUTARATE">
Use of FORMULA in the notes element is discouraged, 

Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_pVAL_b "pVAL[b]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_pVAL_b "pVAL[b]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_Cytochromes_C_Reduced_mi "Cytochromes_C_Reduced[mi]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_Cytochromes_C_Reduced_mi "Cytochromes_C_Reduced[mi]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_FattyAcid_b "FattyAcid[b]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_FattyAcid_b "FattyAcid[b]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_GLN_c "GLN">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_GLN_c "GLN">
Use of CHARGE in the notes element is discoura

Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_STARCH_p "STARCH[p]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_L_CITRULLINE_p "L-CITRULLINE">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_L_CITRULLINE_p "L-CITRULLINE">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_CAII_v "CAII[v]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_CAII_v "CAII[v]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_1_KETO_2_METHYLVALERATE_p "1-KETO-2-METHYLVALERATE">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_1_KETO_2_METHYLVALERATE_p "1-KETO-2-METHYLVALERATE">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_L_CITRULLINE_m "L-CITRULLINE">
Use of FORMULA 

Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_DPG_c "DPG">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_DPG_c "DPG">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_Charged_ILE_tRNAs_c "Charged_ILE_tRNAs[c]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_Charged_ILE_tRNAs_c "Charged_ILE_tRNAs[c]">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_GLC_6_P_c "GLC-6-P">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_GLC_6_P_c "GLC-6-P">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_1_L_MYO_INOSITOL_1_P_c "1-L-MYO-INOSITOL-1-P">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_1_L_MYO_INOSITOL_1_P_c "1-L-MYO-INOSITOL-1-P">
Use of CHARGE in the n

Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_NITRITE_c "NITRITE">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_NITRITE_c "NITRITE">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_NADP_c "NADP">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_NADP_c "NADP">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_ARG_v "ARG">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_ARG_v "ARG">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_2_ACETO_LACTATE_p "2-ACETO-LACTATE">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_2_ACETO_LACTATE_p "2-ACETO-LACTATE">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_GDP_D_GLUCOSE_

...

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_NH4_tx "NH4_tx">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_ADENYLOSUCCINATE_SYNTHASE_RXN_p "ADENYLOSUCCINATE-SYNTHASE-RXN:adenylosuccinate synthetase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_RXN_3523_p "RXN-3523_p">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_Mg_biomass "Mg_biomass">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_FERRIC_CHELATE_REDUCTASE_RXN_c "FERRIC-CHELATE-REDUCTASE-RXN:Ferric-chelate reductase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_OAA_xc "OAA_xc">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_ARGININE_TRNA_LIGASE_RXN_c "ARGININE--TRNA-LIGASE-RXN:Arginine--tRNA ligase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_CYSTAGLY_RXN_p "CYSTAGLY-RXN:Cystathionine &amp;gamma;-lyase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_GLY3KIN_RXN_p "GLY3KIN-RXN:Glycerate kinase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_FORMYLTHFDEFORMYL_RXN_p "FORMYLTHFDEFORMYL-RXN:Formyltetrahydrofolate deformylase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PHOSGLYPHOS_RXN_p "PHOSGLYPHOS-RXN:Phosphoglycerate kinase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_2_PERIOD_7_PERIOD_7_PERIOD

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_GLUC1PURIDYLTRANS_RXN_c "GLUC1PURIDYLTRANS-RXN:UTP--glucose-1-phosphate uridylyltransferase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_MYO_INOSITOL_OXYGENASE_RXN_c "MYO-INOSITOL-OXYGENASE-RXN:Myo-inositol oxygenase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_CYSTATHIONINE_BETA_LYASE_RXN_p "CYSTATHIONINE-BETA-LYASE-RXN:Cystathionine &amp;beta;-lyase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_6PFRUCTPHOS_RXN_p "6PFRUCTPHOS-RXN:NIL">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PRIBFAICARPISOM_RXN_p "PRIBFAICARPISOM-RXN:N-(5&#039;-phospho-D-ribosylformimino)-5-amino-1-(5&#039;&#039;-phosphoribosyl)-4- imidazole carboxamide isomer

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PYRROLINECARBREDUCT_RXN_NADP_c "PYRROLINECARBREDUCT-RXN">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PYRROLINECARBREDUCT_RXN_NAD_c "PYRROLINECARBREDUCT-RXN">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_unlProtHYPO_c "unlProtHYPO_c">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_HS_pc "plastidic HS diffusion">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_ATP_CITRATE_PRO_S_LYASE_RXN_c "ATP-CITRATE-PRO-S--LYASE-RXN:ATP-citrate lyase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_TRP_pc "TRP_pc">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBoun

internal reactions in Shameer core AT model:  637
unbalanced rxns in Shameer AT model:  313
percent unbalanced:  49.13657770800628


In [26]:
shameer_model.reactions.Plastidial_ATP_Synthase_p

Reaction identifier,Plastidial_ATP_Synthase_p
Name,Plastidial_ATP_Synthase_p
Memory address,0x07fb85c6b9910
Stoichiometry,2.4 ADP_p + 14.0 PROTON_l + 3.0 Pi_p + 0.6 aADP_p --> 2.7 ATP_p + 11.3 PROTON_p + 3.0 WATER_p + 0.3 aATP_p 2.4 ADP + 14.0 PROTON + 3.0 Pi[p] + 0.6 aADP[p] --> 2.7 ATP + 11.3 PROTON + 3.0 WATER + 0.3 aATP[p]
GPR,
Lower bound,0.0
Upper bound,1000.0


In [27]:
for items in unbalanced_rxns:
    print(items.id, items.check_mass_balance())

1_PERIOD_2_PERIOD_1_PERIOD_9_RXN_c {'charge': -2.0, 'H': -2.0}
SPONTPRO_RXN_m {'charge': -1.0, 'H': -1.0}
RXN0_5224_c {'charge': -1.0, 'H': -1.0}
RXN_9527_p {'charge': 1.0, 'H': 1.0}
HISTOLDEHYD_RXN_p {'charge': -1.65, 'H': -1.6499999999999995}
MYO_INOSITOL_OXYGENASE_RXN_c {'charge': -1.0, 'H': -1.0}
ATPPHOSPHORIBOSYLTRANS_RXN_p {'charge': -0.6500000000000006, 'H': -0.6500000000000021, 'O': -2.220446049250313e-15, 'P': -3.3306690738754696e-16}
GLUC1PURIDYLTRANS_RXN_c {'charge': 0.8499999999999996, 'H': 0.8499999999999979, 'P': 2.220446049250313e-16}
ORNCARBAMTRANSFER_RXN_p {'charge': -1.0, 'H': -1.0}
RXN_9661_p {'charge': 1.0, 'H': 1.0}
1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mc {'charge': -2.0, 'H': -2.0}
pPHE_biomass {'charge': 2.0, 'H': 4.0, 'N': 3.0, 'O': 1.0, 'A': 3.0}
RXN_6883_mi {'charge': -2.0, 'H': -2.0}
Protein_Polymerisation_c {'charge': -1.0999999999999996, 'H': -1.1000000000000032, 'O': -1.7763568394002505e-15, 'N': 1.0, 'A': 1.0}
Pi_PROTON_mc {'charge': 0.30000000000000004, 'H'

In [28]:
shameer_model.reactions.Plastidial_ATP_Synthase_p

Reaction identifier,Plastidial_ATP_Synthase_p
Name,Plastidial_ATP_Synthase_p
Memory address,0x07fc549709ee0
Stoichiometry,2.4 ADP_p + 14.0 PROTON_l + 3.0 Pi_p + 0.6 aADP_p --> 2.7 ATP_p + 11.3 PROTON_p + 3.0 WATER_p + 0.3 aATP_p 2.4 ADP + 14.0 PROTON + 3.0 Pi[p] + 0.6 aADP[p] --> 2.7 ATP + 11.3 PROTON + 3.0 WATER + 0.3 aATP[p]
GPR,
Lower bound,0.0
Upper bound,1000.0


In [29]:
shameer_model.reactions.RXN_7678_NADP_p

Reaction identifier,RXN_7678_NADP_p
Name,RXN-7678_p
Memory address,0x07fc54965b220
Stoichiometry,CHLOROPHYLL_B_p + NADPH_p + PROTON_p --> CPD_7016_p + NADP_p CHLOROPHYLL-B_p + NADPH + PROTON --> CPD-7016_p + NADP
GPR,
Lower bound,0.0
Upper bound,1000.0


Notes:
Weirdly enough the Shameer model is even much more imbalanced compared dun sa current model ko, which I believe is more of an encoding issue as opposed with actual mass-charge violations, considering that there weren't much changes done by the author in their supplementary scripts. 
Meanwhile, the core Arabidopsis model used by B&B doesn't have much mass-charge violations owing to the small size of the model.
So far after implementing constraints on cofactor recycling (based on the Boolean logic algorithm formulated by Lakshmanan in the original iOS2164) there seems to be limited evidence of any type II or type III internal loops with pFBA. Furthermore, there is concordance with the scale of flux values when comparing with the experimentation by Shameer et al in both the purely linear diel model and the hybrid kinetic setup (2018 and 2019), indicating that the model produces relatively realistic ATP rates and other things. 